In [3]:
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle

In [4]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [5]:
from pandas_profiling import ProfileReport

In [6]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [7]:
data=pd.read_csv('train.csv')

In [8]:
data.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
data.shape

(20800, 5)

In [10]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

** We can see multiple null values here but for text type we can't impute the missing values so we will drop the missing values **

In [11]:
data=data.dropna()
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [12]:
data.shape

(18285, 5)

In [13]:
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [14]:
messages=data.copy()

** I am resetting the index because some missing values are deleted so the entire row will be deleted so that index also get deleted ***

In [15]:
messages.reset_index(inplace=True,drop=True)

In [16]:
messages.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
7,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
8,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
9,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [17]:
#profile =ProfileReport(messages)

In [18]:
#profile.to_file('output.html')

# Data processing

In [19]:

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re


In [20]:
from nltk.corpus import brown
brown.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [21]:

ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    

In [22]:
corpus[3]

'civilian kill singl us airstrik identifi'

In [23]:
sentence_length=[]
for i in range(0,len(messages)):
    x=messages['title'][i].split()
    sentence_length.append(len(x))
    

In [24]:
maxlen=max(sentence_length)

In [25]:
voc_size=5000


In [26]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1283 4641 4227]
 [   0    0    0 ... 2503 2952 1137]
 [   0    0    0 ... 4329  174  681]
 ...
 [   0    0    0 ...  770 4398 1067]
 [   0    0    0 ... 1337 3091   21]
 [   0    0    0 ... 3243 3934 3902]]


In [27]:

## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
y=messages['label']

In [29]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [32]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 12s 990us/sample - loss: 0.3367 - accuracy: 0.8412 - val_loss: 0.1968 - val_accuracy: 0.9173
Epoch 2/10
12250/12250 [==============================] - 7s 595us/sample - loss: 0.1386 - accuracy: 0.9447 - val_loss: 0.2069 - val_accuracy: 0.9213
Epoch 3/10
12250/12250 [==============================] - 8s 614us/sample - loss: 0.0973 - accuracy: 0.9654 - val_loss: 0.2641 - val_accuracy: 0.9205
Epoch 4/10
12250/12250 [==============================] - 7s 552us/sample - loss: 0.0717 - accuracy: 0.9744 - val_loss: 0.2467 - val_accuracy: 0.9095
Epoch 5/10
12250/12250 [==============================] - 8s 654us/sample - loss: 0.0467 - accuracy: 0.9836 - val_loss: 0.2744 - val_accuracy: 0.9034
Epoch 6/10
12250/12250 [==============================] - 7s 602us/sample - loss: 0.0327 - accuracy: 0.9889 - val_loss: 0.3734 - val_accuracy: 0.9138
Epoch 7/10
12250/12250 [==========================

In [33]:
y_pred=model.predict_classes(X_test)

In [34]:
from sklearn import metrics 
metrics.accuracy_score(y_test,y_pred)

0.9047224523612262

In [35]:
metrics.f1_score(y_test,y_pred)

0.8900363358194684